# Projeto final - SME0130 Redes Complexas

**Professor Francisco Rodrigues**

- **Arthur Vergaças Daher Martins | 12542672**
- **Gustavo Sampaio Lima | 12623992**
- **João Pedro Duarte Nunes | 12542460**
- **Pedro Guilherme dos Reis Teixeira | 12542477**

## Tópico escolhido:

<p style="text-align: center; font-size: 24px">3 – Como a cooperação é influenciada pela topologia da rede?</p>


In [1]:
import numpy as np
import networkx as nx
import random
from copy import deepcopy
from enum import Enum, auto
from typing import Self, Callable

In [2]:
class Strategy(Enum):
    COOPERATOR = auto()
    DEFECTOR = auto()

In [3]:
class UpdateRule(Enum):
    REP = auto()
    UI = auto()
    MOR = auto()

    def __str__(self) -> str:
        match self:
            case self.REP:
                return "Replicator Dynamics"
            case self.UI:
                return "Unconditional Imitation"
            case self.MOR:
                return "Moran rule"

    def update(
        self,
        player_node: int,
        players: list,
        network: nx.Graph,
        temptation_to_defect_payoff: float,
        handle_chosen_target: Callable,
    ):
        player = players[player_node]
        neighbors_indexes = [n for n in network.neighbors(player_node)]

        if len(neighbors_indexes) == 0:
            handle_chosen_target(player)
            return

        match self:
            case UpdateRule.REP:
                chosen_neighbor_index = random.choice(neighbors_indexes)
                chosen_neighbor = players[chosen_neighbor_index]

                player_degree = network.degree[player_node]  # type: ignore
                chosen_neighbor_degree = network.degree[chosen_neighbor_index]  # type: ignore

                probability_to_change = (
                    (chosen_neighbor.current_payoff - player.current_payoff)
                    / temptation_to_defect_payoff
                    * max(player_degree, chosen_neighbor_degree, 1)
                )

                if random.random() > probability_to_change:
                    handle_chosen_target(chosen_neighbor)
                else:
                    handle_chosen_target(player)

            case UpdateRule.UI:
                neighbors = [players[n] for n in neighbors_indexes]

                best_neighbor = max(neighbors, key=lambda n: n.current_payoff)

                if best_neighbor.current_payoff > player.current_payoff:
                    handle_chosen_target(best_neighbor)
                else:
                    handle_chosen_target(player)

            case UpdateRule.MOR:
                neighbors_indexes = [players[n] for n in neighbors_indexes]

                neighbors_and_player = [*neighbors_indexes, player]

                sum_of_payoffs = sum([p.current_payoff for p in neighbors_and_player])

                weights = [
                    p.current_payoff / max(sum_of_payoffs, 1)
                    for p in neighbors_and_player
                ]

                if all([w == 0 for w in weights]):
                    handle_chosen_target(player)
                    return

                chosen_target = random.choices(neighbors_and_player, weights=weights)[0]

                handle_chosen_target(chosen_target)

In [21]:
def random_subset(seq: list, m: int, probabilities: None|list = None):
    """
    Return m elements of a sequence, probabilities for each element can de defined, if not uniform distribution is assumed.

    Differently from the usual np.random.choice de use of a set garantees that no repeated elements will be returned.
    """

    targets = set()
    while len(targets) < m:
        x = np.random.choice(seq, p=probabilities)
        targets.add(x)
    return targets

def normalize_prob(probabilities: list):
    """
    Given a list of probabilities they are normalized so that their sum is equal to 1
    """
    full_sum = sum(probabilities)
    normalized_probabilities = list(map(lambda x: x/full_sum, probabilities))
    return normalized_probabilities


def non_linear_barabasi_albert_graph(n, m, power=1, initial_graph=None):
    """
    Generates a random non-linear Barabasi Albert Graph. 
    
    If power > 1 it generates a Superlinear Barabasi Albert Graph
    If power < 1 it generates a Sublinear Barabasi Albert Graph
    If power = 1 it generates a Linear Barabasi Albert Graph

    If no power is given it generates a Linear Barabasi Albert Graph
    """
    
    if m < 1 or m >= n: raise nx.NetworkXError(f"Barabási–Albert network must have m >= 1 and m < n, m = {m}, n = {n}")
    if power <= 0: raise nx.NetworkXError(f"Power must not be zero or negative, power = {power}")

    if initial_graph is None:
        # Default initial graph : star graph on (m + 1) nodes
        G = nx.star_graph(m)
    else:
        if len(initial_graph) < m or len(initial_graph) > n:
            raise nx.NetworkXError(
                f"Barabási–Albert initial graph needs between m={m} and n={n} nodes"
            )
        G = initial_graph.copy()

    # Lists existing nodes, their degrees and their preference in the attachment according to the choosen power
    nodes = list(G.nodes)
    degrees = []
    probabilities = []
    for _, degree in sorted(G.degree(), key=lambda pair: pair[0]):
        degrees.append(degree)
        probabilities.append(pow(degree, power))
    
    # Start adding the other n - m0 nodes.
    source = len(G)
    while source < n:
        
        # Chooses m unique nodes from the existing ones following the preferential attachment
        targets = random_subset(nodes, m, normalize_prob(probabilities))
        # Add edges to m nodes from the source.
        G.add_edges_from(zip([source] * m, targets))
        
        for target in targets:
            degrees[target] += 1  # Increases the degree of each node which received a new connection
            probabilities[target] = pow(degrees[target], power)  # Gets the new preference for the node according to their new degree
        
        nodes.append(source)  # Adds source to the list of nodes
        degrees.append(m)  # Adds the degree of source to the list of degrees
        probabilities.append(pow(m, power))  # Adds the preference of source to the list of probabilites

        source += 1
    return G

In [601]:
class NetworkType(Enum):
    ER = auto()
    WS_0 = auto()
    WS_005 = auto()
    WS_01 = auto()
    WS_03 = auto()
    BA = auto()
    SUB_BA = auto()
    SUPER_BA = auto()

    def __str__(self) -> str:
        match self:
            case self.ER:
                return "Erdős-Rényi Network Type"
            case self.WS_0:
                return "Watts-Strogatz (Small-World) Network Type (p = 0)"
            case self.WS_005:
                return "Watts-Strogatz (Small-World) Network Type (p = 0.05)"
            case self.WS_01:
                return "Watts-Strogatz (Small-World) Network Type (p = 0.1)"
            case self.WS_03:
                return "Watts-Strogatz (Small-World) Network Type (p = 0.3)"
            case self.BA:
                return "Barabási-Albert Network Type"
            case self.SUB_BA:
                return "Sublinear Barabási-Albert Network Type"
            case self.SUPER_BA:
                return "Superlinear Barabási-Albert Network Type"

    def generator_function(
        self, number_of_nodes: int, average_degree: float
    ) -> Callable[[], nx.Graph]:
        match self:
            case self.ER:
                return lambda: nx.fast_gnp_random_graph(
                    number_of_nodes, average_degree / (number_of_nodes - 1)
                )
            case self.WS_0:
                return lambda: nx.watts_strogatz_graph(
                    number_of_nodes, int(average_degree), 0
                )
            case self.WS_005:
                return lambda: nx.watts_strogatz_graph(
                    number_of_nodes, int(average_degree), 0.05
                )
            case self.WS_01:
                return lambda: nx.watts_strogatz_graph(
                    number_of_nodes, int(average_degree), 0.1
                )
            case self.WS_03:
                return lambda: nx.watts_strogatz_graph(
                    number_of_nodes, int(average_degree), 0.3
                )
            case self.BA:
                return lambda: nx.barabasi_albert_graph(
                    number_of_nodes, int(average_degree / 2)
                )
            case self.SUB_BA:
                return (
                    lambda: non_linear_barabasi_albert_graph
                    (number_of_nodes, int(average_degree / 2), power=0.5)
                )  
            case self.SUPER_BA:
                return (
                    lambda: non_linear_barabasi_albert_graph
                    (number_of_nodes, int(average_degree / 2), power=2)
                )

In [602]:
class Player:
    def __init__(self, strategy: Strategy, update_rule: UpdateRule):
        self.strategy = strategy
        self.update_rule = update_rule
        self.current_payoff = 0

    @classmethod
    def from_fractions(
        cls,
        cooperators_fraction: float,
        rule_a_fraction: float,
        update_rules: list[UpdateRule],
    ) -> Self:
        strategy = random.choices(
            [Strategy.COOPERATOR, Strategy.DEFECTOR],
            weights=[cooperators_fraction, 1 - cooperators_fraction],
        )[0]

        update_rule = random.choices(
            update_rules, weights=[rule_a_fraction, 1 - rule_a_fraction]
        )[0]

        return cls(strategy, update_rule)

    def play_with(
        self,
        other: Self,
        temptation_to_defect_payoff: float,
        mutual_cooperation_payoff: float,
    ):
        # Seguindo o dilema do prisioneiro fraco (Nowak and May), que pode ser descrito da seguinte forma:
        #
        #    | C | D |
        #  C | x | 0 |
        #  D | b | 0 |
        #
        # Onde as linhas indicam a estratégia do jogador, e as colunas a estratégia do seu adversário.
        # Usualmente, x = 1.
        #

        if (
            self.strategy == Strategy.COOPERATOR
            and other.strategy == Strategy.COOPERATOR
        ):
            self.current_payoff += mutual_cooperation_payoff
        elif (
            self.strategy == Strategy.DEFECTOR and other.strategy == Strategy.COOPERATOR
        ):
            self.current_payoff += temptation_to_defect_payoff

    def evolve(
        self,
        self_node: int,
        players: list[Self],
        network: nx.Graph,
        temptation_to_defect_payoff: float,
    ):
        self.update_rule.update(
            self_node,
            players,
            network,
            temptation_to_defect_payoff,
            lambda target: self._copy(target),
        )

    def reset(self):
        self.current_payoff = 0

    def _copy(self, other: Self):
        self.strategy = other.strategy
        self.update_rule = other.update_rule

    def __repr__(self) -> str:
        return f"PLAYER: {self.strategy} | {self.update_rule} | payoff={self.current_payoff}"

In [659]:
def simulate_prisoners_dilemma(
    number_of_iterations=4000,
    number_of_players=2500,
    temptation_payoff=1.0,
    mutual_cooperation_payoff=1.0,
    cooperators_fraction=0.5,
    rule_a_fraction=0.5,
    update_rules=[UpdateRule.MOR, UpdateRule.UI],
    network_type: NetworkType = NetworkType.BA,
    average_network_degree=6,
):
    cooperators_fraction_at_iteration = []
    rule_a_fraction_at_iteration = []

    def store_network_snapshot():
        number_of_cooperators = 0
        number_of_rule_a = 0
        for node in network:
            player = players[node]

            number_of_cooperators += player.strategy == Strategy.COOPERATOR
            number_of_rule_a += player.update_rule == update_rules[0]

        cooperators_fraction_at_iteration.append(
            number_of_cooperators / number_of_players
        )
        rule_a_fraction_at_iteration.append(number_of_rule_a / number_of_players)

    network = network_type.generator_function(
        number_of_players, average_network_degree
    )()

    players: list[Player] = [
        Player.from_fractions(cooperators_fraction, rule_a_fraction, update_rules)
        for _ in range(number_of_players)
    ]

    store_network_snapshot()
    for _ in range(number_of_iterations):
        # jogar contra os vizinhos
        for node in network:
            player = players[node]
            neighbors = [players[n] for n in network.neighbors(node)]

            for adversary in neighbors:
                player.play_with(
                    adversary, temptation_payoff, mutual_cooperation_payoff
                )

        # evoluir de acordo com vizinhos
        players_snapshot = [
            deepcopy(p) for p in players
        ]  # copiar estado dos jogadores para que a evolução ocorra em paralelo

        for node in network:
            player = players[node]
            player.evolve(node, players_snapshot, network, temptation_payoff)

        store_network_snapshot()

        # resetar payoffs para próxima iteração
        for player in players:
            player.reset()

    return (
        np.array(cooperators_fraction_at_iteration),
        np.array(rule_a_fraction_at_iteration),
    )

In [650]:
def simulate_prisoners_dilemma_for_different_b(
    b_start_value=1.0,
    b_final_value=2.0,
    b_iterations=20,
    number_of_iterations=4000,
    number_of_players=2500,
    mutual_cooperation_payoff=1.0,
    cooperators_fraction=0.5,
    rule_a_fraction=0.5,
    update_rules=[UpdateRule.MOR, UpdateRule.UI],
    network_type: NetworkType = NetworkType.BA,
    average_network_degree=6,
):
    cooperators_fraction_at_end_of_simulation = []
    rule_a_fraction_at_end_of_simulation = []

    for b in np.linspace(b_start_value, b_final_value, b_iterations):
        cooperators_fractions, rule_a_fractions = simulate_prisoners_dilemma(
            number_of_iterations=number_of_iterations,
            number_of_players=number_of_players,
            temptation_payoff=b,
            mutual_cooperation_payoff=mutual_cooperation_payoff,
            cooperators_fraction=cooperators_fraction,
            rule_a_fraction=rule_a_fraction,
            update_rules=update_rules,
            network_type=network_type,
            average_network_degree=average_network_degree,
        )

        cooperators_fraction_at_end_of_simulation.append(cooperators_fractions[-1])
        rule_a_fraction_at_end_of_simulation.append(rule_a_fractions[-1])

    return (
        np.array(cooperators_fraction_at_end_of_simulation),
        np.array(rule_a_fraction_at_end_of_simulation),
    )

In [662]:
cooperators_fractions, rule_a_fractions = simulate_prisoners_dilemma(
    number_of_iterations=100,
    average_network_degree=6,
    temptation_payoff=1.5,
    update_rules=[UpdateRule.REP, UpdateRule.UI],
    network_type=NetworkType.ER,
)

print(cooperators_fractions)
print(rule_a_fractions)

[0.5088 0.4388 0.4156 0.4192 0.4412 0.4796 0.5112 0.5212 0.5432 0.57
 0.584  0.5936 0.6052 0.6236 0.6364 0.6468 0.6656 0.6692 0.668  0.6648
 0.664  0.6612 0.6568 0.6544 0.6532 0.6496 0.6436 0.6356 0.6292 0.6264
 0.6236 0.6228 0.6168 0.6128 0.6064 0.6004 0.6008 0.596  0.594  0.5884
 0.5868 0.584  0.5796 0.5764 0.5744 0.5736 0.5716 0.5696 0.5668 0.5644
 0.5624 0.5608 0.5572 0.556  0.5536 0.5528 0.55   0.5452 0.5448 0.544
 0.5412 0.5384 0.5384 0.5364 0.5364 0.5364 0.5336 0.532  0.5288 0.5304
 0.5296 0.53   0.5276 0.5268 0.5268 0.5272 0.5276 0.5272 0.528  0.526
 0.526  0.5256 0.5244 0.5252 0.522  0.5232 0.5232 0.522  0.522  0.5204
 0.5208 0.5196 0.52   0.5176 0.518  0.518  0.5184 0.5148 0.516  0.5148
 0.514 ]
[0.4948 0.5704 0.6144 0.6528 0.688  0.708  0.7128 0.7272 0.7264 0.7256
 0.7312 0.7316 0.7336 0.7372 0.7404 0.7432 0.738  0.7348 0.7284 0.7208
 0.7152 0.7104 0.7032 0.7032 0.6988 0.692  0.6808 0.6708 0.6644 0.6624
 0.6584 0.654  0.6468 0.6412 0.6356 0.6304 0.6296 0.6232 0.6196 0.6156
 

In [651]:
cooperators_fractions, rule_a_fractions = simulate_prisoners_dilemma_for_different_b(
    b_start_value=1.0,
    b_final_value=2.0,
    b_iterations=50,
    number_of_iterations=100,
    average_network_degree=6,
    update_rules=[UpdateRule.REP, UpdateRule.UI],
    network_type=NetworkType.WS_03,
)

print(cooperators_fractions)
print(rule_a_fractions)

[0.9212 0.9292 0.9316 0.89   0.9164 0.8704 0.8864 0.9116 0.898  0.8484
 0.5472 0.5928 0.5864 0.4732 0.5044 0.4488 0.4968 0.442  0.4352 0.4324
 0.422  0.4408 0.4828 0.4448 0.4388 0.4144 0.3816 0.3948 0.4028 0.396
 0.4392 0.4636 0.4988 0.432  0.4064 0.408  0.4468 0.3932 0.4492 0.4336
 0.4144 0.4804 0.432  0.4744 0.4416 0.4544 0.4396 0.4332 0.392  0.384 ]
[0.1256 0.1232 0.1348 0.188  0.1984 0.266  0.2044 0.2032 0.146  0.1668
 0.5152 0.2992 0.3444 0.3512 0.3444 0.3736 0.3344 0.3772 0.402  0.3988
 0.4236 0.442  0.4828 0.4308 0.4336 0.4144 0.368  0.3768 0.3912 0.3888
 0.4336 0.4652 0.4988 0.4348 0.4072 0.4084 0.4488 0.3964 0.452  0.4344
 0.4152 0.4812 0.4316 0.4752 0.4384 0.4564 0.4412 0.434  0.3932 0.3844]
